In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf  
import seaborn as sns
import sys

In [ ]:
#load data
log_returns = pd.read_csv('DATA/ETF_log_returns.csv', index_col='Date', parse_dates=True)
adj_close_price = pd.read_csv('DATA/ETF_ADJ_close_returns.csv', index_col='Date', parse_dates=True)
# Check it loaded:
# log_returns
# close_returns

In [ ]:
#assign etf vector
etfs = ['DIA','SPY','QQQ']

In [ ]:
def ma_crossover_strategy( #creating the ma_crossover_strategy function to calculate returns based on paramaters and 
    etfs,
    log_returns,
    short_ma,
    long_ma,
    threshold=0.0,  #input paramaters
    initial_position=0  # NEW: carry over position from training
):
    log_return_summary = {}  # empty dictionary to store returns

    for etf in etfs: #run the following for each etf: 
        # Build the DataFrame for this ETF with necessary columns
        df = pd.DataFrame({ #make a dataframe of the following dictionary key-value pairs
            'log_return': log_returns[etf],
            'short_ma': short_ma[etf],
            'long_ma': long_ma[etf]
        }).dropna()  # Drop rows with any missing data

        # ENTRY signal: short MA crosses above long MA + threshold
        df['entry_signal'] = (
            (df['short_ma'] > df['long_ma'] * (1 + threshold)) &
            (df['short_ma'].shift(1) <= df['long_ma'].shift(1))
        )

        # EXIT signal: short MA crosses below long MA - threshold
        df['exit_signal'] = (
            (df['short_ma'] < df['long_ma'] * (1 - threshold)) &
            (df['short_ma'].shift(1) >= df['long_ma'].shift(1))
        )

        # Initialize position tracking column
        df['position'] = np.nan

        # Apply entry/exit rules
        df.loc[df['entry_signal'], 'position'] = 1
        df.loc[df['exit_signal'], 'position'] = 0

        # Forward-fill position column, starting with initial_position
        df['position'] = df['position'].ffill()
        df.loc[df.index[0], 'position'] = initial_position  # Set first value manually 
        df['position'] = df['position'].ffill().fillna(0)

        # Shift position to avoid lookahead bias
        df['shifted_position'] = df['position'].shift(1)
        
        # Drop rows with NA from shift
        df.dropna(inplace=True)

        # Compute strategy returns
        df['strategy_log_return'] = df['shifted_position'] * df['log_return']

        # Count trades
        trade_count = df['entry_signal'].sum()

        # Store strategy return (Buy & Hold done outside now)
        log_return_summary[etf] = {
            'Total Strategy Return': np.exp(df['strategy_log_return'].sum()) - 1,
            'Trade Count': int(trade_count)
        }

    return pd.DataFrame(log_return_summary).T


In [ ]:
def corrected_loop_with_buffer(adj_close_price, log_returns, etfs, train_years=2, test_years=1):
    #creating a function that will loop long and short_ma combinations on various thresholds to find optimal returns on training years 
    #and then compare performance on testing years. Walk forward split 2yrs-training and then 1yr-testing for entireity of the dataset
    walk_results = []
    #creating an empty list to store results

    # Threshold search space
    #creating a vector list of thresholds. from 0 to .005 jumping by .00025 and rounding to 5 decimal points
    thresholds = np.round(np.arange(0.0, 0.005, 0.00025), 5)

    # Get full date range
    #finding the entirity date length of the dataset using index of adjusted closing prices(row labels - dates0
    full_dates = adj_close_price.index #will give all the dates
    start_year = full_dates.min().year #will find the minimum date - the earliest date - furthest back
    end_year = full_dates.max().year #will find the latest date in the dataset

    for start in range(start_year, end_year - train_years - test_years + 1): #creating a range  (x-y) for each starting year 
        #meaning a range for 1999-2022 and then for 2000 - 2022 and so on... end year is subtracted by training years and test years but we 
        #add one because range(a,b) the a is inclusive and b is exclusive meaning not included in the calculation meaning the last year is cutoff
        #basically saying that for every number in this range do the following(exclusive of end value)
        
        # 1. Define calendar-aware train/test windows
        train_start = pd.Timestamp(f"{start}-01-01") #use timestamp from pandas library storing the starting date for each range loop
        #turns the string into a datetime object for arithmetic calculations
        train_end   = train_start + pd.DateOffset(years=train_years) - pd.Timedelta(days=1)
        # the dateoffset is adding exactly training years to the timestamp =2yrs and the timedelta is subtracting 1 day so we land on 12-31
        test_start  = train_end + pd.Timedelta(days=1)
        #the test start adds a day so we land on the 1st of the year again
        test_end    = test_start + pd.DateOffset(years=test_years) - pd.Timedelta(days=1)
        #the test end is getting the test years added which is 1yr and subtracting 1 day again so we land on 12-31
#Use Case	Recommended Function	Why?
# Add/subtract days	pd.Timedelta(days=...)	Exact time math
# Add/subtract weeks	pd.Timedelta(weeks=...)	Exact, consistent
# Add/subtract months	pd.DateOffset(months=...)	Handles month length variation
# Add/subtract years	pd.DateOffset(years=...)	Handles leap years, calendar logic
# logic here is that dateoffset can also subtract but would have to use negative value so visually better to use Timedelta also when subtracting

        # 2. Slice training data
        adj_train = adj_close_price.loc[train_start:train_end]
#slicing data so that we have the adj_train assigned to the rows from training start to training end. both inclusive 
        log_train = log_returns.loc[train_start:train_end]
        #getting log_test
        log_test = log_returns.loc[test_start:test_end]
        #getting adj_test
        adj_test = adj_close_price.loc[test_start:test_end]


        # 3. Grid search for best strategy parameters
        results = [] #creating an empty list for results for the best short and long_ma combination
        for short in range(5, 50): # Loop through all short moving average windows from 5 to 49 (inclusive)
            for long in range(short + 1, 51):# For each short MA do a loop through x+1 to 50 and do the following for each
 
                short_ma = adj_train.rolling(short).mean() #create a rolling average
                long_ma = adj_train.rolling(long).mean()
                short_df = pd.DataFrame(short_ma, index=adj_train.index, columns=adj_train.columns) #create a dataframe (ensure consinstency)
                long_df = pd.DataFrame(long_ma, index=adj_train.index, columns=adj_train.columns)

                for threshold in thresholds: 
# For every short_ma value from 5 to 49,
# try every possible long_ma value that is larger than short_ma (from short_ma+1 to 50).
# For each (short_ma, long_ma) pair, loop through all threshold values from 0.0 to 0.005, stepping by 0.00025.
# This way, we try every combination of short MA, long MA, and threshold to find the best-performing set.

                    print(f"Training Year: {start}, Short: {short}, Long: {long}, Threshold: {threshold}", end="\r")
                    sys.stdout.flush() #end="\r"tells Python not to move to a new line after printing.next print() to overwrite the same line
                    #the sys.stdout.flush() basically forces out output meaning it gets rid of any buffer or anything that python might have to print output

                    summary = ma_crossover_strategy(
                        etfs, log_train, short_df, long_df, threshold
                    ) #run this combination through our strategy function define above 

                    for etf in etfs: #and for every etf defined earlier as ['DIA,'SPY','QQQ'] append adding an element to the end of a list and add the following 
                        results.append({ #{ = dict.
                            'ETF': etf,  #defining dictionary terms meaning we are adding a dictionary into a list with key 'ETF' (STRING) and value of the ETF = 'QQQ'
                            'Short_MA': short,
                            'Long_MA': long,
                            'Threshold': threshold,
                            'Return': summary.loc[etf, 'Total Strategy Return'],
                            'Trade Count': summary.loc[etf, 'Trade Count']
                        })

        # 4. Select best config per ETF
        results_df = pd.DataFrame(results) #turning  the giant dictionary into a dataframe. keys become columns and values become rows
        best_ma = results_df.sort_values('Return', ascending=False).groupby('ETF').first() 
        #now we sort by return and then group by ETF and then take the first row for eahc which is the highest return 
        #this is inside the training year loop so it is doing this for every individual training period 

        # 5. Apply best strategy on the test set and apply a buffer
        buffer_days = 100 #assigning buffer to 100 days 
        test_start_buffered = test_start - pd.Timedelta(days=buffer_days) #subtracting buffer days from test set to go back 100 days 
        adj_test_buffered = adj_close_price.loc[test_start_buffered:test_end] #snip the following rows from adjusted close from 100 back from test period to end of test period

        for etf in etfs: #now for the best paramaters for that testing period for each etf do the following 
            short = int(best_ma.loc[etf, 'Short_MA']) #assign short_ma value to short variable
            long = int(best_ma.loc[etf, 'Long_MA'])
            threshold = best_ma.loc[etf, 'Threshold']

            # Rolling averages on buffered data
            short_ma = adj_test_buffered[etf].rolling(short).mean() #calculate the rolling average using paramaters from testing period but applying 
            #to the buffered testing period (includes 100 days back)
            long_ma = adj_test_buffered[etf].rolling(long).mean() #same for long. this buffer ensures that our logn and short are calcualted before testing period so we dont have x empty data
            #rows and going in blind(unrealistic when historical data is available) 

            short_df = pd.DataFrame(short_ma, index=adj_test_buffered.index, columns=[etf]) #wrap those into a df again ensuring consistent columns and index
            long_df  = pd.DataFrame(long_ma, index=adj_test_buffered.index, columns=[etf])

            # Determine last signal from training set
            last_train_short = adj_train[etf].rolling(short).mean().iloc[-1]  #finding the last value of short in training period
            last_train_long = adj_train[etf].rolling(long).mean().iloc[-1]    #finding the last value of long in training period 
            initial_position = int(last_train_short > last_train_long * (1 + threshold)) #calculating the boolean (TRUE or FALSE) but the 
            #int() is turning the TRUE=1 and FALSE=0. so if the last short was greater than the last long multiplied by optimal threshold then 
            #we are ina  position

            # Run strategy on the TESTING SET now with previous position signal 
            summary = ma_crossover_strategy(
                etfs=[etf],
                log_returns=log_test,
                short_ma=short_df,
                long_ma=long_df,
                threshold=threshold,
                initial_position=initial_position  # Must be supported in your function
            )

            # Store results (test_start buffer already handled inside strategy)
            walk_results.append({ #adding new element to list, specifically adding dictionaries(key-value pair)
                'Train Start': train_start,
                'Train End': train_end,
                'Test Start': test_start,
                'Test End': test_end,
                'ETF': etf,
                'Short_MA': short,
                'Long_MA': long,
                'Threshold': threshold,
                'Strategy Return': round(summary.loc[etf, 'Total Strategy Return'], 4),
                'Trade Count': int(summary.loc[etf, 'Trade Count'])
            })

    return pd.DataFrame(walk_results) #returning the results as a dataframe 


In [ ]:
 # Example call
results_df = corrected_loop_with_buffer(
     adj_close_price=adj_close_price,      # your full adjusted close price DataFrame
     log_returns=log_returns,              # your full log return DataFrame
     etfs=['DIA', 'SPY', 'QQQ'],           # list of ETFs you want to backtest
     train_years=2,                        # training window length
     test_years=1                          # testing window length
)

In [ ]:
results_df.to_csv("DATA/position_buffer_results.csv", index=False)
print("Saved to walk_forward_results.csv ✅ HURRAY")

# # To get average strategy return per ETF:
# walk_df.groupby('ETF')[['Strategy Return', 'Buy & Hold Return', 'Trade Count']].mean().round(4)



In [ ]:
results_df

#boxplot of strategy returns thorughout time by ETF

plt.figure(figsize=(10, 6))
sns.boxplot(data=results_df, x='ETF', y='Strategy Return')
plt.title("Distribution of Strategy Returns by ETF")
plt.grid(True)
plt.show()


In [ ]:
 # Bar Plot of Average Strategy Return vs Buy & Hold (per ETF)
avg_returns = results_df.groupby('ETF')['Strategy Return'].mean().reset_index()

avg_returns.plot(
    x='ETF',
    kind='bar',
    figsize=(8, 5),
    title='Average Returns: Strategy vs Buy & Hold',
    rot=0
)
plt.ylabel("Average Return")
plt.grid(True)
plt.show()
